# RisePro Triage
> Config Extractor for PrivateLoader

- toc: true 
- badges: true
- categories: [risepro,stealer,config,triage]

## Overview

According to [FlashPoint](https://flashpoint.io/blog/risepro-stealer-and-pay-per-install-malware-privateloader/https://flashpoint.io/blog/risepro-stealer-and-pay-per-install-malware-privateloader/) 
>“RisePro” is a newly identified stealer written in C++ that appears to possess similar functionality to the stealer malware “Vidar.” RisePro targets potentially sensitive information on infected machines and attempts to exfiltrate it in the form of logs. “RisePro” is a newly identified stealer written in C++ that appears to possess similar functionality to the stealer malware “Vidar.” RisePro targets potentially sensitive information on infected machines and attempts to exfiltrate it in the form of logs. 

RisePRO triggered a false positive for the [Malpedia PrivateLoader yara rule](https://malpedia.caad.fkie.fraunhofer.de/details/win.privateloader) and triggered the UnpacMe PrivateLoader config extractor. This led to correct string extraction from RisePRO even thought it is a separate malware. Are these related?!


## References 
- [“RisePro” Stealer and Pay-Per-Install Malware “PrivateLoader”](https://flashpoint.io/blog/risepro-stealer-and-pay-per-install-malware-privateloader/)
- [PrivateLoader Triage -  Config Extractor for PrivateLoader](https://research.openanalysis.net/privateloader/loader/config/triage/2022/09/08/privateloader.html)
- [privateloader_str_decrypt.py](https://gist.github.com/andretavare5/66ec413cdb4c7c39d35c22d38c7067a8#file-privateloader_str_decrypt-py)
- [Malpedia yara](https://malpedia.caad.fkie.fraunhofer.de/details/win.privateloader)
- [PrivateLoader: the loader of the prevalent ruzki PPI service](https://blog.sekoia.io/privateloader-the-loader-of-the-prevalent-ruzki-ppi-service/)
- [Example string encryption using xmm](https://github.com/JustasMasiulis/xorstrhttps://github.com/JustasMasiulis/xorstr)

## Sample
- `2cd2f077ca597ad0ef234a357ea71558d5e039da9df9958d0b8bd0efa92e74c9`[UnpacMe](https://www.unpac.me/results/31bbf769-34dc-4d9c-af52-721c61853713?hash=2cd2f077ca597ad0ef234a357ea71558d5e039da9df9958d0b8bd0efa92e74c9#/)

## Analysis

Both PrivateLoader and RisePRO use the same plaintext user agent string `Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36` and request header `"Content-Type: application/x-www-form-urlencoded"` also the string encryption algorithm is the same (xmm registers and xor intrinsic). Funny enough this causes a 100% overlap between the PrivateLoader Yara rule and RisePro.




### Yara Rule

PrivateLoader yara rule matches on both PrivateLoader and RisePRO!

```
rule win_privateloader_w0 {
  meta:
    author =    "andretavare5"
    org =       "BitSight"
    date =      "2022-06-06"
    md5 =       "8f70a0f45532261cb4df2800b141551d"
    reference = "https://tavares.re/blog/2022/06/06/hunting-privateloader-pay-per-install-service"
    license =   "CC BY-NC-SA 4.0"
    
    malpedia_reference = "https://malpedia.caad.fkie.fraunhofer.de/details/win.privateloader"
    malpedia_version = "20220824"
    malpedia_license = "CC BY-NC-SA 4.0"
    malpedia_sharing = "TLP:WHITE"
  strings:
    $code = { 66 0F EF (4?|8?) } // pxor xmm(1/0) - str chunk decryption
    $str =  "Content-Type: application/x-www-form-urlencoded" wide ascii
    $ua1 = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" wide ascii
    $ua2 = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36" wide ascii
                              
  condition:
    uint16(0) == 0x5A4D and // MZ
    $str and
    any of ($ua*) and
    #code > 100
}
```

We need new Yara rules for both PrivateLoader and RisePRO to differentiate the two. 

### C2
The C2 domain is stored in plain text in RisePRO unlike PrivateLoader (where it is encrypted). 
```
194.169.175[.]128
```


### String Extraction

The string encryption is almost identical to privateloader, we had to adjust for registers that store immediates for multiple strings in a row. Also, there are situations where the stack string DWORDs are not combined in order. Currently we don't handle this, but the solution is to use the `.dis` memory displacement to determine the order of the the DWORDs. 

We also used Dumpulator for the global STD::Strings build with the CRT initialization functions. This was due to some custom decryption that was not caught by our string decryption algorithm. 



In [2]:
def unhex(hex_string):
    import binascii
    if type(hex_string) == str:
        return binascii.unhexlify(hex_string.encode('utf-8'))
    else:
        return binascii.unhexlify(hex_string)

def tohex(data):
    import binascii
    if type(data) == str:
        return binascii.hexlify(data.encode('utf-8'))
    else:
        return binascii.hexlify(data)


In [38]:
import pefile
import struct
from capstone import *
from capstone.x86 import *

SAMPLE_PATH = '/tmp/bad.bin'


def is_ascii(s):
    return all(c < 128 or c == 0 for c in s)


def get_reg_data(instructions, reg_name):
    search_count = 0
    search_limit = 2000
    for inst in instructions:
        
        if search_count > search_limit:
            break
        search_count += 1
        
        if  inst.mnemonic == 'mov' and  inst.operands[0].type == X86_OP_REG  and  inst.operands[1].type == X86_OP_IMM:
            if inst.reg_name(inst.operands[0].reg) == reg_name:
                imm_value = inst.operands[1].value.imm
                data_chunk = struct.pack('<I',imm_value)
                return data_chunk
    return None


def get_data(instructions):
    data_chunks = []
    count = 0
    steps = 0
    steps_flag = 0
    flag_reg = 0
    search_count = 0
    search_limit = 400
    
    for inst_ptr in range(0,len(instructions)):
        inst = instructions[inst_ptr]
        steps +=1
        
        if search_count > search_limit:
            break
        search_count += 1
         
        if  inst.mnemonic == 'call':
            break

        if  inst.mnemonic == 'mov' and  inst.operands[0].type == X86_OP_REG  and  inst.operands[1].type == X86_OP_IMM:
            flag_reg = 1
        
        if  inst.mnemonic == 'mov' and inst.operands[0].type == X86_OP_MEM and inst.operands[1].type == X86_OP_REG:
            reg_name = inst.reg_name(inst.operands[1].reg)
            #print(f"Scanning for {reg_name} data....")
            result = get_reg_data(instructions[inst_ptr:], reg_name)
            if result is None:
                #print("scanning failed")
                break
            #print(f"Found reg {reg_name} data {result}")
            data_chunks.append(result)
            count += 1
            steps = 0
            steps_flag = 1
         
        if  inst.mnemonic == 'mov' and ( (inst.operands[0].type == X86_OP_MEM and inst.operands[0].value.mem.disp != 0) or inst.operands[0].type == X86_OP_REG ) and inst.operands[1].type == X86_OP_IMM:
            imm_value = inst.operands[1].value.imm
            #print(hex(imm_value))
            if imm_value & 0xff000000 == 0:
                break
            data_chunk = struct.pack('<I',imm_value)
            data_chunks.append(data_chunk)

            count += 1
            steps = 0
            steps_flag = 1

        if steps == 16 and steps_flag:
            break
         #if steps == 6 and steps_flag:  # if you got some garbage string use this instead of the above
            #break
       
    
    enc_data = data_chunks[0:count//2][::-1]
    key = data_chunks[count//2:count][::-1]
    
    if flag_reg :
        enc_data = sum(zip(enc_data[1::2], enc_data[::2]), ())
        key = sum(zip(key[1::2], key[::2]), ())
    return b''.join(enc_data),b''.join(key)


        
filename = SAMPLE_PATH

# disassemble .txt section
pe = pefile.PE(filename)
md = Cs(CS_ARCH_X86, CS_MODE_32) 
md.detail = True
md.skipdata = True
addr = 0
instructions = []
txt = pe.sections[0]


# TODO: we don't seem to be disassembling the full section?!!
image_base = pe.OPTIONAL_HEADER.ImageBase
section_rva = txt.VirtualAddress

for inst in md.disasm(txt.get_data(), image_base + section_rva):
    instructions.append(inst)


# search, build and decrypt strings
strings = []
addr = None
string = ''

for i, inst in enumerate(instructions):

    if inst.mnemonic == 'pxor':
    # if inst.address == 0x0045C8A0:
        #print(hex(inst.address))
        #try: # possible string decryption found  
        reversed_instruction_list = instructions[:i][::-1]
        encrypted_str, key = get_data(reversed_instruction_list)
        # print(f"str_len: {len(encrypted_str)}, key_len: {len(key)}")
        # print(encrypted_str.hex())
        # print(key.hex())
        
        if len(encrypted_str) == 0 or len(key) == 0:
            #print(f"Error at {hex(inst.address)} key or data is missing")
            continue
        
        if len(encrypted_str) != len(key):
            #print(f"Error at {hex(inst.address)} key and data not equal length")
            continue

        out = bytearray(encrypted_str[j] ^ key[j] for j in range(len(key)))
        
        #print(out)
        out = out.replace(b'\x00',b'')
        if len(out) == 0:
            continue
        #print(out.decode('utf-8'))
        
        if is_ascii(out):
            strings.append((inst.address,out.decode('utf-8')))

                           
# print(len(strings))
                           
for s in strings:
    print(f'{hex(s[0])} {s[1]}')

#enc string
# D2 EA F6 5F
# 9A 2F 1D 5C               
# 3B A0 36 4E                
# F0 81 B8 52                 
 
# key
# B7 92 86 36
# E8 4E 69 35
# 54 CE 69 37
# 95 E0 CA 52   
    
print("done")

0x459fe0 \FileZilla
0x45a0eb gins
0x45a526 IndexedDB
0x45a823 lets
0x45ac41 IndexedDB
0x45afb0 tory\His
0x45bf57 name_on_card
0x45c051 card_number
0x45c0f5 last_four
0x45c176 last_four
0x45c555 exp_month
0x45c8a0 expiration_year
0x45e560 expirationDate
0x45f3d4 expirationDate
0x45fb81 wb
0x4622e5 bhghoamapcdpbohphigoooaddinpkbai
0x4624d3 nkbihfbeogaeaoehlefnkodbefgpgknn
0x462909 kncchdigobghenbbaddojjnnaogfppfj
0x462b0c fihkakfobkmkjojpchpfgcmhfjnmnfpi
0x462d0f nkddgncdjgjfcddamfgcmfnlhccnimig
0x462edb nanjmdknhkinifnkgdcggcfnhdaammmj
0x462ef9 nanjmdknhkinifnkgdcggcfnhdaammmj
0x462f12 nanjmdknhkinifnkgdcggcfnhdaammmj
0x463109 nlbmnnijcnlegkjjpcfjclmcfggfefdm
0x463318 amkmjjmmflddogmhpjloimipbofnfjih
0x46351b nhnkbkgjikgcigadomkphalanndcapjk
0x463721 cphhlgmgameodnhkjdmkpanlelnlohao
0x463924 fnjhmkhhmkbjkkabndcnnogagogbneec
0x463b27 kpfopkelmapcoipemfendmdcghnegimn
0x463d2a blnieiiffboillknjnepogjhkgnoapac
0x463f2d hpglfhgfnhbgpjdenjgmdgoeiappafln
0x4640f9 hnfanknocfeofbddgcijnmhnfnkdna

In [23]:

string_start = 0x0045C1D3
test_addr = 0x0045C8A0 
test_rva = test_addr - image_base
test_offset = pe.get_offset_from_rva(test_rva)

test_addr - string_start


1741

In [44]:
def ida_to_dmp_rva(ida_addr, image_base, dump_image_base):
    ida_rva = ida_addr - image_base
    return ida_rva + dump_image_base

In [60]:
from dumpulator import Dumpulator

dp = Dumpulator("/tmp/rise.dmp", quiet=True)

main_addr = 0x00458C9B
main_rva = main_addr - image_base
dump_image_base = dp.modules.main
main_dump_addr = main_rva + dump_image_base


dp.start(dp.regs.rip, end=main_dump_addr)
print("done")



def read_std_string(dp, addr):
    str_out = b''
    str_size = dp.read_long(addr + 16)
    if str_size > 15:
        str_ptr = dp.read_ptr(addr)
        str_data = dp.read(str_ptr, str_size)
        return str_data
    else:
        str_out = dp.read_str(addr)
    return str_out
    

global_str_addr = ida_to_dmp_rva(0x004F06B0, image_base, dp.modules.main)
print(read_std_string(dp, global_str_addr))

global_str_addr = ida_to_dmp_rva(0x004F06E0, image_base, dp.modules.main)
config = read_std_string(dp, global_str_addr)

    
global_str_addr = ida_to_dmp_rva(0x04F07DC, image_base, dp.modules.main)
print(read_std_string(dp, global_str_addr))

commit(0x6b6000[0x1000], PAGE_READWRITE)
commit(0x6b7000[0x2000], PAGE_READWRITE)
commit(0x6b9000[0x2000], PAGE_READWRITE)
done
50500
bytearray(b'RisePro\r\nTelegram: https://t.me/RiseProSUPPORT')


'0xc60000'